# NHL MVP Analysis

This project is a first foray into machine learning. The goal is to predict player's vote share of the Hart trophy, which is awarded to the most valuable player in the National Hockey League as voted on by members of the Professional Hockey Writer's Association.  

Hockey statistics come from Evolving Hockey and Hockey-Reference. At the moment only basic statistics provided in the Hart trophy candidates by year summary page from Hockey-Reference are used. A medium term goal is to scrape data including advanced statistics to combine with the existing data. As it will turn out, prediction performance based only on the basic statistics used to far is not very good. 

References/Acknowledgements:
Most of what appears in this notebook was learned/borrowed from Kaggle. In particular, two specific notebooks I heavily referenced are https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python and https://www.kaggle.com/mmueller/stacking-starter.

In [3]:
# imports
import numpy as np
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import os
import time
import csv 
import sklearn
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
import warnings

import xgboost as xgb

from sklearn.ensemble import (RandomForestRegressor,  ExtraTreesRegressor)
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.model_selection import (KFold, train_test_split, cross_val_score)
from sklearn.metrics import (mean_absolute_error)
from sklearn.pipeline import make_pipeline

from xgboost import XGBRegressor
from bs4 import BeautifulSoup


warnings.filterwarnings('ignore')
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

%matplotlib inline
py.init_notebook_mode(connected=True)
pd.set_option('display.max_columns', 500)
os.environ['KMP_DUPLICATE_LIB_OK']='True'


ImportError: No module named xgboost

In [ ]:
try:
    full_data= pd.read_csv("hockey-data-hart.csv")    
except:
    # Scrape data
    # specify the url
    years = range(1936,2018 + 1)
    addresses = ["https://www.hockey-reference.com/awards/voting-{}.html#all-hart-stats".format(yr) for yr in years]
    names = ['Year',  
             'player',
             'age',
             'team_id',
             'pos',
             'votes',
             'pct_of_vote',
             'first',
             'second',
             'third',
             'fourth',
             'fifth',
             'goals',
             'assists',
             'points',
             'plus_minus',
             'wins_goalie',
             'losses_goalie',
             'ties_goalie',
             'goals_against_avg',
             'save_pct',
             'ops',
             'dps',
             'gps',
             'ps']

    with open('hockey-data-hart.csv', 'w') as csvfile:    #Create the csv file
        hockeywriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        hockeywriter.writerow(names)
        for i, pg in enumerate(addresses):
            time.sleep(0.1) #pause the code for a sec
            r = requests.get(pg)
            # parse the html using beautiful soup and store in variable `soup`
            soup = BeautifulSoup(r.text, "html.parser")
            values = [years[i]]  #first value

            #Go through the stats and add them to a list
            for tag in soup.findAll("td"):
                if tag.string == 'None': #don't save 'None' string
                    values.append('') 
                elif tag['data-stat']=='team_id': #use proper team name
                    try: #though it may not exist
                        values.append(tag.contents[0]['title'])
                    except:
                        values.append(tag.string)
                else:
                    values.append(tag.string)
                if tag['data-stat']=='ps': #if last value in row start on next line
                    hockeywriter.writerow(values)
                    values = [years[i]]  #set first value again     

In [ ]:
full_data= pd.read_csv("hockey-data-hart.csv")    


# clean it up
full_data=full_data.dropna(axis=0,subset=['pct_of_vote'])
full_data=full_data.drop(['first','second','third','fourth','fifth'],axis=1)
full_data = full_data[full_data["pct_of_vote"]>0]

# let's also drop goalies who are missing save pct, and skaters who are missing plus_minus
full_data = full_data.loc[~((full_data.pos=="G") & full_data.save_pct.isna())]
full_data = full_data.loc[~((full_data.pos!="G") & full_data.plus_minus.isna())]

# remaining na's should be due to mismatched position with stats. set them to zero
full_data = full_data.fillna(0)

# for non goalies (goalies), let's replace all goalie (non goalie) stats by zero rather than NaN

# Create a new feature p_simp: for players who have joint positions, we just keep the first one
def pos_combine(name):
    title_search = re.search('(^[A-Z]{1,2})', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""

full_data["pos"] = full_data["pos"].apply(pos_combine)
full_data = full_data[full_data["pct_of_vote"]>1]

skaters = full_data[full_data['pos']!='G']
goalies = full_data[full_data['pos']=='G']
   
full_data.tail(20)

In [ ]:
print(full_data.describe())
full_data['pct_of_vote'].plot.hist()

In [ ]:
fig = plt.figure()
sns.kdeplot(skaters['goals'])
fig = plt.figure()
sns.jointplot(x='goals',y='pct_of_vote',data = skaters, kind='kde', gridsize=40)

In [ ]:
# examining some skater data
sns.pairplot(skaters[['plus_minus','pct_of_vote','age','goals', 'assists','ps']])

In [ ]:
# examining some goalie data
sns.pairplot(goalies[['pct_of_vote','age','wins_goalie','losses_goalie','ties_goalie','goals_against_avg','save_pct','ps']])

In [ ]:
g = sns.FacetGrid(skaters,col="pos")
g.map(sns.scatterplot, 'age','points')

In [ ]:
fd = skaters
features= ['pct_of_vote','goals','assists','plus_minus','age','ops','dps','pos']
fd = fd[features]
f = fd.corr()
plt.figure(figsize=(len(features),len(features)))

sns.heatmap(f,annot=True,cmap="RdBu_r",vmin=-1,vmax=1);

In [ ]:
fd = goalies
features= ['pct_of_vote','age','wins_goalie','losses_goalie',
           'ties_goalie','goals_against_avg','save_pct','ps']
fd = fd[features]
f = fd.corr()
plt.figure(figsize=(len(features),len(features)));

sns.heatmap(f,annot=True,cmap="RdBu_r",vmin=-1,vmax=1);

We can expect from the plots above that directly fitting a model to the available data might be challenging. Nonetheless we will try anyways.

In [ ]:
fd = full_data.copy()
# prepare for applying model:
# 1) preprocessing-scale
# t = fd.drop(["Year"],axis=1).select_dtypes(exclude=["object"]).apply(sklearn.preprocessing.scale)
# fd[t.columns] = t
# actually its incorrect to preprocess the entire data set. this results in data contamination


# 2) one-hot encode positions
fd=pd.get_dummies(fd,columns=['pos'])

fd[fd["Year"]==2018]
# full_data.sample(10)
# fd.pct_of_vote.plot.hist()


In [ ]:


model_features = [ u'age', u'goals', u'assists',  u'plus_minus',
       u'wins_goalie', u'losses_goalie', u'ties_goalie', u'goals_against_avg',
       u'save_pct', u'ops', u'dps', u'gps', u'pos_C',
       u'pos_D', u'pos_G', u'pos_LW', u'pos_RW']

# Use all years 2015 and prior for training, and try to predict the previous three years.
target_yr = 2015
train=fd[fd["Year"]<=target_yr]
test=fd[fd["Year"] > target_yr]


# Prepare a holdout of the train set to use for blending. 
base, holdout = train_test_split(train,test_size=0.1)

x_test = test[model_features]
y_test = test.pct_of_vote

x_base = base[model_features]
y_base = base.pct_of_vote

x_holdout = holdout[model_features]
y_holdout = holdout.pct_of_vote


# x_base and y_base will be used to fit the base models
# then make predictions for y_holdout using x_holdout
# those predictions are used to train the stacker




# # scaling preprocessing the data. I'm not sure if it's actually a good idea to do this
# # Leave it out for now.

# t = x_base.drop(["pos_G"],axis=1).select_dtypes(exclude=["object"]).apply(sklearn.preprocessing.scale)
# x_base[t.columns] = t

# t = x_holdout.drop(["pos_G"],axis=1).select_dtypes(exclude=["object"]).apply(sklearn.preprocessing.scale)
# x_holdout[t.columns] = t

# t = x_test.drop(["pos_G"],axis=1).select_dtypes(exclude=["object"]).apply(sklearn.preprocessing.scale)
# x_test[t.columns] = t

In [ ]:
# Wrapper class for sklearn models
# It trains on logarithmically renormalized vote_pct, rather than raw.
# This is because heuristically the vote share by player
# in any given year is approximately exponentially decaying
# so it made sense to me to try to make the data more regular by taking the log.
class SklearnWrapper(object):
    def __init__(self,clf,seed=0,params=None):
        params['random_state'] = seed
        self.clf = clf(**params)
        
        
    # automatically does log conversion in train and predict
    def train(self,x_train,y_train):
        self.clf.fit(x_train,np.log(y_train))
        
    def predict(self,x):
        return np.exp(self.clf.predict(x))
    
    def feature_importances(self):
        print(clf.feature_importances_)
        
def get_base_predictions(clf):

    clf.train(x_base,y_base)
    return clf.predict(x_holdout)
    
# Wrapper for xgboost from sklearn courtesy of https://www.kaggle.com/mmueller/stacking-starter
class XgbWrapper(object):
    def __init__(self, seed=0, params=None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 250)

    def train(self, x_train, y_train):
        dtrain = xgb.DMatrix(x_train, label=np.log(y_train))
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds)

    def predict(self, x):
        return np.exp(self.gbdt.predict(xgb.DMatrix(x)))
        



In [ ]:
#Try several models

# Put in our parameters. For now these values are the same as those in 
# https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python

# Random Forest parameters
et_params = {
    'n_jobs': -1,
    'n_estimators': 100,
    'max_features': 0.5,
    'max_depth': 12,
    'min_samples_leaf': 2,
}

rf_params = {
    'n_jobs': -1,
    'n_estimators': 100,
    'max_features': 0.6,
    'max_depth': 12,
    'min_samples_leaf': 2,
}


# Gradient Boosting parameters
gb_params = {
    'n_estimators': 500,
     'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}



mlp_params = {
    'hidden_layer_sizes': (4,4)
}

SEED = 0

# Train two versions of each model, one on vote_pct, and one on log_vote_pct
# for convenience, also generate predictions for the test data to use later

forest = SklearnWrapper(clf=RandomForestRegressor,seed=SEED,params=rf_params)
# print("Forest Regressor MAE on training data: %f"% mean_absolute_error(forest.predict(x_holdout),y_holdout))

gbm = SklearnWrapper(clf=XGBRegressor,seed=SEED,params=gb_params)
# print("XGBRegressor MAE on training data: %f"% mean_absolute_error(gbm.predict(x_holdout),y_holdout))

mlp = SklearnWrapper(clf=MLPRegressor,seed=SEED,params=mlp_params)
# print("MLPRegressor MAE on training data: %f"% mean_absolute_error(mlp.predict(x_holdout),y_holdout))

et = SklearnWrapper(clf=ExtraTreesRegressor,seed=SEED,params=et_params)
# print("Extra Trees Regressor MAE on training data: %f"% mean_absolute_error(et.predict(x_holdout),y_holdout))


In [ ]:
# use XGBoost to blend the base models
    
lvl1_predictions = pd.DataFrame()

lvl1_predictions["rf"] = get_base_predictions(forest)
lvl1_predictions["xgb"] = get_base_predictions(gbm)
# lvl1_predictions["svr"] = get_base_predictions(svr)
lvl1_predictions["et"] = get_base_predictions(et)
lvl1_predictions["mlp"] = get_base_predictions(mlp)



# look at correlation map of base predictions
plt.figure()
zz = lvl1_predictions.corr()
sns.heatmap(zz,annot=True,cmap="RdBu_r",vmin=-1,vmax=1);

# the models are decently correlated with each other. 
# this is not ideal, but that's life

print(lvl1_predictions.head())


In [ ]:
# train the ensemble 
xgb_params = {
    'seed': 0,
    'colsample_bytree': 0.7,
    'silent': 1,
    'subsample': 0.7,
    'learning_rate': 0.075,
    'max_depth': 7,
    'num_parallel_tree': 1,
    'min_child_weight': 1,
    'eval_metric': 'mae',
    'nrounds': 350
}

stack = XgbWrapper(seed=SEED,params=xgb_params)

stack.train(lvl1_predictions,y_holdout)

# feature importances, out of curiosity
# print zip(lvl1_predictions.columns,stack.feature_importances)


# prepare input predictions for blender
x_test_lvl2 = pd.DataFrame(columns = {
                                          })

x_test_lvl2["rf"] = forest.predict(x_test)
x_test_lvl2["xgb"] = gbm.predict(x_test)
# x_test_lvl2["svr"] = svr_test_F
x_test_lvl2["et"] = et.predict(x_test)
x_test_lvl2["mlp"] = mlp.predict(x_test)





lvl2_prediction = stack.predict(x_test_lvl2)


print("Averaged MAE %f" % mean_absolute_error(lvl2_prediction,y_test))

test["final_prediction"] = lvl2_prediction


# fig = plt.figure()



test.plot.scatter(x='pct_of_vote',y='final_prediction')

fig = plt.figure(figsize=(16,16))

# g = sns.FacetGrid(test,col="Year",height=6,orientation="h")
# g.map(sns.barplot, 'player','pct_of_vote')
# sns.catplot(x='player', y = 'pct_of_vote',data=test,kind='bar',row='Year',sharex=False,height=14)


# its annoyingly difficult to make a facetgrid to show the three years
with sns.axes_style("darkgrid"):
    for year in [2016,2017,2018]:
        t = test[test['Year']==year]
        fig = plt.figure()
        t[['player','pct_of_vote','final_prediction']].head(10).plot.bar(x='player')
        plt.title("Year %i" % year)

    

# life is hard


Some years, such as 2018, are particularly unkind. After a brief investigation, our model seems to strongly overweigh goals, which is not an unreasonable thing to do. The model just sees the statistics in a vacuum, and has no idea about the team's record during that year, or advanced statistics such as WAR that more directly quantify how useful a player was to their team. So there is clearly room for improvement here.

In [ ]:
#Make the plotting nicer by only using the last names
surnames = [re.search('\w+ (\w+)', name).group(1) for name in test["player"]]
test["player_surname"] = surnames

In [ ]:
#Showing all three years together
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(21, 7), sharex=True)

# sns.barplot(x='pct_of_vote', y='final_prediction', data=test[test.loc[:,"Year"]==2017], palette="vlag", ax=ax2)
sns.barplot(x='player_surname', y='pct_of_vote', data=test, palette="deep", ax=ax1)
ax2.axhline(0, color="k", clip_on=False)
ax2.set_ylabel("% of Vote")

# sns.barplot(x='player', y='final_prediction', data=test[test.loc[:,"Year"]==2018], palette="deep", ax=ax3)
sns.barplot(x='player_surname', y='final_prediction', data=test, palette="deep", ax=ax2)
ax3.axhline(0, color="k", clip_on=False)
ax3.set_ylabel("Predicted % of Vote")

# Finalize the plot
sns.despine(bottom=True)
plt.setp(f.axes, yticks=[])
plt.tight_layout(h_pad=2)